# Using External Data
---
# If you fork, do leave an upvote!

In [ ]:
# Imports
import numpy as np
import pandas as pd
import time 
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Using Point Spread Data
Credit goes to Dave Lorenz. Check out his github repo and leave a star there as well
https://github.com/DaveLorenz/NCAAMarchMadnessNN

In [ ]:
spread_df = []
years_list = [20, 19, 18,17,16,15,14,13,12,11,10,'09','08','07','06','05','04','03']
year_int = 2021
for year in years_list: 
    temp_spread_df = pd.read_csv("../input/moredata2/Data/Point spreads/ncaabb" + str(year) + ".csv")
    temp_spread_df['Season'] = year_int
    year_last = year
    year_int = year_int-1    
    if year==20:
        spread_df = temp_spread_df
    else:
        spread_df = spread_df.append(temp_spread_df)
spread_df['home'] = spread_df['home'].str.lower()
spread_df['road'] = spread_df['road'].str.lower()

# Match Team IDs to data

teams_df = pd.read_csv('../input/ncaam-march-mania-2021/MTeamSpellings.csv', sep='\,', engine='python', encoding='cp1252')

teams_df.head()

team_list = ['home','road']
for team in team_list:
    spread_df = pd.merge(spread_df, teams_df, left_on=[team], right_on = ['TeamNameSpelling'], how='left')
    if team=='home':
        spread_df.rename(columns={'TeamID': 'HTeamID'}, inplace=True)
    if team=='road':
        spread_df.rename(columns={'TeamID': 'RTeamID'}, inplace=True)
    spread_df = spread_df.drop(['TeamNameSpelling'], axis=1)


spread_df.loc[spread_df['hscore']>spread_df['rscore'], 'WTeamID'] = spread_df['HTeamID']
spread_df.loc[spread_df['hscore']<spread_df['rscore'], 'LTeamID'] = spread_df['HTeamID']

spread_df.loc[spread_df['hscore']>spread_df['rscore'], 'WScore'] = spread_df['hscore']
spread_df.loc[spread_df['hscore']<spread_df['rscore'], 'LScore'] = spread_df['hscore']

spread_df.loc[spread_df['rscore']>spread_df['hscore'], 'WTeamID'] = spread_df['RTeamID']
spread_df.loc[spread_df['rscore']<spread_df['hscore'], 'LTeamID'] = spread_df['RTeamID']

spread_df.loc[spread_df['rscore']>spread_df['hscore'], 'WScore'] = spread_df['rscore']
spread_df.loc[spread_df['rscore']<spread_df['hscore'], 'LScore'] = spread_df['rscore']

spread_df.replace([np.inf, -np.inf], np.nan).dropna(subset=['WTeamID','LTeamID','WScore','LScore','line'])
spread_df = spread_df[spread_df['WScore'] != "."]
spread_df = spread_df[spread_df['LScore'] != "."]
spread_df = spread_df[spread_df['line'] != "."]

spread_df['line'].value_counts(dropna=False)

#drop spreads with bad coverage
spread_df = spread_df.drop(['line7ot','lineargh','lineash','lineashby','linedd','linedunk','lineer','linegreen','linemarkov','linemass','linepib','linepig','linepir','linepiw','linepom','lineprophet','linerpi','lineround','linesauce','lineseven','neutral','lineteamrnks','linetalis','lineespn','linemassey','linedonc','linesaggm','std','linepugh','linefox','linedok','lineopen'], axis=1)

spread_df.tail()

spread_df.dropna(subset=['line'])
spread_df

## Using Kenpom Data

In [ ]:
kp_df = []
years_list = [2021, 2020, 2019,2018,2017,2016,2015,2014,2013,2012,2011,
              2010,2009,2008,2007,2006,2005,2004,2003,2002]

for year in years_list:
    temp_kp_df = pd.read_csv("../input/moredata2/Data/Ken Pom/KP" + str(year) + "_final.csv",  index_col=False)
    temp_kp_df=temp_kp_df.rename(columns={'adj_o': 'adjo', 'adj_d': 'adjd', 'conference': 'conf'})
    temp_kp_df['Season'] = year
    year_last = year   
    if year==2021:
        kp_df = temp_kp_df
    else:
        kp_df = kp_df.append(temp_kp_df)

    kp_df['Season'].value_counts(dropna=False)

kp_df['team'] = kp_df['team'].str.lower()

# Match Team IDs to data

teams_df = pd.read_csv('../input/ncaam-march-mania-2021/MTeamSpellings.csv', sep='\,', engine='python', encoding='cp1252')
teams_df.head()

kp_df = pd.merge(kp_df, teams_df, left_on=['team'], right_on = ['TeamNameSpelling'], how='left')
kp_df = kp_df.drop(['TeamNameSpelling'], axis=1)

kp_df['Season'].value_counts(dropna=False)
kp_df.head()

# Write the data to a csv
kp_df.to_csv('kenpom.csv', index=False)
kp_df = kp_df.drop(['Unnamed: 0', 'adjem'], 1)
kp_df.to_csv('kenpom.csv', index=False)
kp_df

In [ ]:
spread_df.to_csv('./spread.csv')

## Combining Data

In [ ]:

# Load packages

import numpy as np
import pandas as pd


# Combine all tournament and reg season games

tourney_compact_df = pd.read_csv('../input/ncaam-march-mania-2021/MNCAATourneyCompactResults.csv')
tourney_compact_df['is_tourney'] = 1.0

regseason_compact_df = pd.read_csv('../input/ncaam-march-mania-2021/MRegularSeasonCompactResults.csv')
regseason_compact_df['is_tourney'] = 0.0

all_games_df = regseason_compact_df.append(tourney_compact_df)
all_games_df.head()

all_games_df.tail()


# Load and merge on point spread

spread_df = pd.read_csv('./spread.csv')
spread_df = spread_df[['Season','date','line','lineavg','road','home','rscore','hscore','WScore','LScore','HTeamID','RTeamID','WTeamID','LTeamID']]

# Merge point spread on to games

all_games_df = pd.merge(all_games_df, spread_df, left_on=['WTeamID','LTeamID','Season','WScore','LScore'], right_on = ['WTeamID','LTeamID','Season','WScore','LScore'], how='inner')
all_games_df = all_games_df[['Season','date','is_tourney','home','HTeamID','hscore','road','RTeamID','rscore','line','lineavg']]

all_games_df['Hwin'] = 0
all_games_df.loc[all_games_df['hscore']>all_games_df['rscore'], 'Hwin'] = 1      

all_games_df

# # Add seeds

seeds_df = pd.read_csv('../input/ncaam-march-mania-2021/MNCAATourneySeeds.csv')
seeds_df['Seed_num'] = seeds_df['Seed'].str.extract('(\d\d)', expand=True)
seeds_df['Seed_num'] = pd.to_numeric(seeds_df['Seed_num'])
seeds_df = seeds_df[['Season','TeamID','Seed_num']]
seeds_df.rename(columns={'Seed_num': 'Seed'}, inplace=True)
seeds_df

#Note: we merge twice for this merge and subsequent merges, because the data are in a wide format by team (i.e., there are two variables rather than two rows for each game)
home_road = ['H','R']
for hr in home_road:
    all_games_df = pd.merge(all_games_df, seeds_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    all_games_df.rename(columns={'Seed': hr+'Seed'}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID'], axis=1)

all_games_df


kp_df = pd.read_csv('./kenpom.csv')

efficiency_list = ['conf','adjem','adjo','adjd','luck']
for hr in home_road:
    all_games_df = pd.merge(all_games_df, kp_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='inner')
    for metric in efficiency_list:
        all_games_df.rename(columns={metric: hr+metric}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID','team'], axis=1)



# Add Massey and Ken Pom rankings

massey_df = pd.read_csv('../input/ncaam-march-mania-2021/MMasseyOrdinals.csv')
massey_df21 = massey_df.loc[massey_df['Season'] == 2021]

#Ranking at the end of season versus end of tourney (as shown in KP data)
POM_df_else = massey_df[massey_df['SystemName'].str.contains("POM")]
POM_end_df_else = POM_df_else.loc[POM_df_else['RankingDayNum'] == 133]
POM_df21 = massey_df21[massey_df21['SystemName'].str.contains("POM")]
POM_end_df_21 = POM_df21.loc[POM_df21['RankingDayNum'] == 128]
POM_end_df = POM_end_df_else.append(POM_end_df_21)
POM_end_df = POM_end_df[['Season','TeamID','OrdinalRank']]
POM_end_df.rename(columns={'OrdinalRank': 'RankPOM'}, inplace=True)

for hr in home_road:
    all_games_df = pd.merge(all_games_df,POM_end_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    all_games_df.rename(columns={'RankPOM': hr+'RankPOM'}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID'], axis=1)

# Calculate regular season avg stats

regseason_detail_df = pd.read_csv('../input/ncaam-march-mania-2021/MRegularSeasonDetailedResults.csv')
regseason_detail_df



#Attach the POM data
teamslist = ['W','L']
for wl in teamslist:
    regseason_detail_df = pd.merge(regseason_detail_df, POM_end_df, left_on=[wl+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    regseason_detail_df.rename(columns={'RankPOM': wl+'RankPOM'}, inplace=True)
    regseason_detail_df = regseason_detail_df.drop(['TeamID'], axis=1)

# Regular season (create one row for each team with avg stats and win info)

win_team_only = regseason_detail_df.drop(['LTeamID','WAst','LAst','LStl','WStl','LBlk','WBlk','LPF','WPF','NumOT','DayNum'], axis=1)
win_team_only.rename(columns={'WTeamName': 'TeamName'}, inplace=True)
win_team_only.rename(columns={'WTeamID': 'TeamID'}, inplace=True)

team_factors_list = ['Score','FGM','FGA','FGA3','FGM3','TO','FTM','FTA','RankPOM']
for factor in team_factors_list:
    win_team_only.rename(columns={'W'+factor: factor}, inplace=True)
    win_team_only.rename(columns={'L'+factor: 'Opponent'+factor}, inplace=True)

win_team_only['wins'] = 1 

win_team_only['wins_top25'] = 0
win_team_only.loc[win_team_only['OpponentRankPOM']<26, 'wins_top25'] = 1

win_team_only['wins_top5'] = 0
win_team_only.loc[win_team_only['OpponentRankPOM']<6, 'wins_top5'] = 1

win_team_only = win_team_only[['Season','TeamID','wins','wins_top25','wins_top5','Score','OpponentScore','FGM','FGA','OpponentFGM','OpponentFGA','FGM3','FGA3','OpponentFGM3','OpponentFGA3','FTM','FTA','OpponentFTM','OpponentFTA','TO','OpponentTO']]

loss_team_only = regseason_detail_df.drop(['WTeamID','WAst','LAst','LStl','WStl','LBlk','WBlk','LPF','WPF','NumOT','DayNum'], axis=1)
loss_team_only.rename(columns={'LTeamName': 'TeamName'}, inplace=True)
loss_team_only.rename(columns={'LTeamID': 'TeamID'}, inplace=True)

for factor in team_factors_list:
    loss_team_only.rename(columns={'L'+factor: factor}, inplace=True)
    loss_team_only.rename(columns={'W'+factor: 'Opponent'+factor}, inplace=True)

loss_team_only['wins'] = 0
loss_team_only['wins_top25'] = 0
loss_team_only['wins_top5'] = 0

loss_team_only = loss_team_only[['Season','TeamID','wins','wins_top25','wins_top5','Score','OpponentScore','FGM','FGA','OpponentFGM','OpponentFGA','FGM3','FGA3','OpponentFGM3','OpponentFGA3','FTM','FTA','OpponentFTM','OpponentFTA','TO','OpponentTO']]

reg_season_all = win_team_only.append(loss_team_only)

reg_season_all['TOmargin'] = reg_season_all['TO']-reg_season_all['OpponentTO']

reg_season_all['PointMargin'] = reg_season_all['Score']-reg_season_all['OpponentScore']

reg_season_all['FG'] = reg_season_all['FGM']/reg_season_all['FGA']
reg_season_all['FGopponent'] = reg_season_all['OpponentFGM']/reg_season_all['OpponentFGA']

reg_season_all['FG3'] = reg_season_all['FGM3']/reg_season_all['FGA3']
reg_season_all['FG3opponent'] = reg_season_all['OpponentFGM3']/reg_season_all['OpponentFGA3']

reg_season_all['FT'] = reg_season_all['FTM']/reg_season_all['FTA']

reg_season_all = reg_season_all.drop(['TO','OpponentTO','FGM','FGA','OpponentFGA','OpponentFGM','FGM3','FGA3','OpponentFGM3','OpponentFGA3','FTM','FTA','OpponentFTM','OpponentFTA'], axis=1)
reg_season_all


# Collapse regular season sums/means

reg_season_means = reg_season_all.groupby(['TeamID','Season']).mean().reset_index()
reg_season_means = reg_season_means.drop(['wins','wins_top25','wins_top5'], axis=1)

reg_season_sum = reg_season_all.groupby(['TeamID','Season']).sum().reset_index()
reg_season_sum = reg_season_sum.drop(['Score','OpponentScore','TOmargin','PointMargin','FG','FG3','FGopponent','FG3opponent','FT'], axis=1)

regseason_df = pd.merge(reg_season_means, reg_season_sum, left_on=['TeamID','Season'], right_on = ['TeamID','Season'], how='left')

regseason_df.to_csv('regseason_df.csv', index=False)


# Combine regular season stats with game level data

#Merge the avg regular season performance with all games and have performance wide: 
performance_list = ['wins','wins_top5','wins_top25','Score','OpponentScore','TOmargin','PointMargin','FG','FGopponent','FG3','FG3opponent','FT','RBMargin']
for hr in home_road:
    all_games_df = pd.merge(all_games_df, regseason_df, left_on=[hr+'TeamID','Season'], right_on = ['TeamID','Season'], how='left')
    for var in performance_list:
        all_games_df.rename(columns={var: hr+var}, inplace=True)
    all_games_df = all_games_df.drop(['TeamID'], axis=1)

# Add binary experience dummy

temp_df = all_games_df[all_games_df['is_tourney'] == 1]
temp_df_home = temp_df[['home']]
temp_df_home.rename(columns={'home': 'team'}, inplace=True)
temp_df_road = temp_df[['road']]
temp_df_road.rename(columns={'road': 'team'}, inplace=True)
tourney_teams_df = temp_df_home.append(temp_df_road)
tourney_teams_df['team'].value_counts(dropna=False)

all_games_df['Htourny20plus'] = 0
all_games_df['Rtourny20plus'] = 0

experienced_teams = ['kansas','north carolina','kentucky','duke','michigan st.','wisconsin','florida','villanova','gonzaga','louisville','arizona','xavier','connecticut','syracuse','butler','ohio st.','ucla','west virginia','texas','michigan','pittsburgh','memphis','oregon']
for team in experienced_teams:
    all_games_df.loc[all_games_df['home']==team, 'Htourny20plus'] = 1
    all_games_df.loc[all_games_df['road']==team, 'Rtourny20plus'] = 1

temp_df = all_games_df[all_games_df['is_tourney'] == 1]
temp_df_home = temp_df[['Hconf']]
temp_df_home.rename(columns={'Hconf': 'conf'}, inplace=True)
temp_df_road = temp_df[['Rconf']]
temp_df_road.rename(columns={'Rconf': 'conf'}, inplace=True)
tourny_teams_df = temp_df_home.append(temp_df_road)
tourny_teams_df['conf'].value_counts(dropna=False)

all_games_df['HBig4Conf'] = 0
all_games_df['RBig4Conf'] = 0
conferences = ['ACC','B10','B12','SEC']
for conf in conferences:
    all_games_df.loc[all_games_df['Hconf']==conf, 'HBig4Conf'] = 1
    all_games_df.loc[all_games_df['Rconf']==conf, 'RBig4Conf'] = 1

#mean imputations of variables for model
all_games_df = all_games_df.fillna(all_games_df.mean())

# Output to csv

all_games_df.to_csv('all_games_df.csv', index=False)
all_games_df.sample(10)

In [ ]:
all_games_df.describe()

---
# Give this notebook an upvote!